# Extend Word2Vec Model with Phonemes 



In [1]:
%load_ext autoreload
%autoreload 2

In [29]:
import math
import random
import sys
import os
from util.Util import *
from util import formspring_data_parser
import pandas as pd
from sklearn import model_selection

In [60]:
import sys
sys.path.append("/Users/luisd/dev/cyberbullying-detection")
from nn import *
import nn 
import fc_nn 

In [4]:
# sys.path.append(...) # in case we want to add something else

In [21]:
import phonemes_from_graphemes as pg
from words_2_vectors import * 
from util import formspring_data_parser

In [25]:
# import importlib
# importlib.reload(pg)
from phonemes_from_graphemes import *

### Where am I expecting to see the data

In [7]:
data_dir = "./data"

In [8]:
common_logger = get_logger(name = "common_logger", debug_log_file_name = "common_logger.log")

2017-06-13 14:27:24,432 - common_logger - INFO - 'common_logger': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger.log'


Logger created
Creating debug handler at '/Users/luisd/dev/cyberbullying-detection/common_logger.log'
'common_logger': logging 'INFO'+ logs to Console, 'DEBUG'+ logs to '/Users/luisd/dev/cyberbullying-detection/common_logger.log'


In [9]:
common_logger.debug("hello, this is a debug msg")

In [10]:
common_logger.handlers[1].baseFilename

'/Users/luisd/dev/cyberbullying-detection/common_logger.log'

In [11]:
#  change 'None' to 'm' to reload 
mw = ModelWrapper.from_google_news_model(data_dir=data_dir, alogger=common_logger)
# mw = ModelWrapper(data_dir=data_dir, alogger = common_logger, m= None, sounds_dict = None)

2017-06-13 14:27:31,328 - common_logger - INFO - Loading model from ./data/GoogleNews-vectors-negative300.bin.gz...
2017-06-13 14:29:59,210 - common_logger - INFO - Model succesfully loaded
2017-06-13 14:29:59,211 - common_logger - INFO - Sort all the words in the model, so that we can auto-complete queries quickly...


In [12]:
m = mw.model # cache model definition

In [13]:
len(m.vocab)

3000000

## Generation of phonemes from graphemes 

Some helper functions are provided to generate sounds from a Word->Vector Model: 
* generate_from_word2vec
* generate_whole_word2vec_vocab 

So, for example, to generate sounds from a subset of the whole data you'd do: 
```
phonemesFactory = PhonemesFromGraphemes(alogger=common_logger)
pg.generate_from_word2vec(mw, phonemesFactory, start = 1, n = 13, a_dir = "delete_this")
```
or to generate the sounds for the WHOLE vocabulary of Word2Vec: 
```
phonemesFactory = PhonemesFromGraphemes(alogger=common_logger)
result_dir = pg.generate_whole_word2vec_vocab(mw, phonemesFactory)
print("All shelves resulting from this generation are individual files under '{}'".format(result_dir))
``` 

## Creation of Sounds' Dictionary  

### Let's read the sounds' dictionary

In [15]:
sounds_dict = SoundsDict(a_dir=data_dir, alogger=common_logger)

In [16]:
sounds_dict['ju:']

{'YOU', 'YOu', 'You', 'you'}

In [18]:
mw.set_sounds_dict(sounds_dict=sounds_dict)

In [19]:
mw.sound_to_word('ju:')

{'YOU', 'YOu', 'You', 'you'}

## Read data from XML  

In [26]:
xml_file_name = '/Users/luisd/Downloads/FormspringLabeledForCyberbullying/XMLMergedFile.xml'
phonemesFactory = PhonemesFromGraphemes(alogger=common_logger)


In [79]:
parser = formspring_data_parser.Formspring_Data_Parser(xml_file_name, pg = phonemesFactory, mw = mw, alogger = mw.alogger)

### Let's parse (or read) the whole document 

* To generate it: `qal = parser.all_questions_answers_labels()` (and then you can save it with `qal.to_csv("a_file_name.csv")`) 
* To read it from somewhere: `all_my_data = pd.read_csv("from_somewhere.csv")`

In [31]:
# all_of_them = parser.questions_answers_labels(an_id = 1)
# all_of_them

In [32]:
# really_all = parser.all_questions_answers_labels()

In [33]:
really_all = pd.read_csv("really_all_of_them.csv")

In [35]:
threatening_items = really_all.loc[really_all["threat"] == True]

In [36]:
threatening_items.shape

(4879, 7)

In [37]:
really_all.shape

(36885, 7)

In [78]:
{'answer', 'question', 'threat'}.issubset(really_all.columns)

True

In [77]:
really_all.columns

Index(['Unnamed: 0', 'answer', 'answer_raw', 'question', 'question_raw',
       'threat', 'uuid'],
      dtype='object')

In [ ]:
# really_all.to_csv("really_all_of_them.csv")

In [38]:
questions_answers = ["{}; {}".format(q, a) for q, a in list(zip(really_all['question'].tolist(), really_all['answer'].tolist()))] 

In [39]:
questions_answers[0:10] 

["what's your favorite song laughter; I like too many songs to have a favorite",
 '----------------------------------------------------------_3; haha jk',
 "'3; haha jk",
 'angel you duh; Really Thanks haha',
 'nan; nan',
 'nan; nan',
 'any makeup tips i suck at doing my makeup laughter; Sure Like tell me wht u know Like wht do you use',
 "I miss It's Emma btw haha; EMMA laughter I yu",
 "I miss It's Emma btw haha; EMMA laughter I ue",
 "I miss It's Emma btw haha; EMMA laughter I ew"]

In [40]:
labels = ["THREAT" if threat else "CLEAN" for threat in really_all['threat'].tolist()]

In [41]:
labels[0:10]

['CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN',
 'CLEAN']

In [42]:
from collections import Counter
import numpy as np

In [43]:
# Create three Counter objects to store positive, negative and total counts
positive_counts = Counter()
negative_counts = Counter()
total_counts = Counter()
for label, review in zip(labels, questions_answers):
    words = review.split(' ') 
    if label == 'THREAT':
        for word in words:
            negative_counts[word] += 1
    else:
        for word in words:
            positive_counts[word] += 1
    for word in words:
        total_counts[word] += 1

pos_neg_ratios = Counter()

unique_words = total_counts.keys()
for word in unique_words:
    if total_counts[word] >= 100:
        pos_neg_ratios[word] = positive_counts[word] / float(negative_counts[word]+1)
        
        
unique_words = pos_neg_ratios.keys()# set(pos_neg_ratios.elements())
for word in unique_words:
    if (pos_neg_ratios[word] > 1):
        pos_neg_ratios[word] = np.log(pos_neg_ratios[word])
    else:
        pos_neg_ratios[word] = -np.log(1/(pos_neg_ratios[word] + 0.01))        

In [44]:
negative_counts.most_common()

[('a', 6752),
 ('you', 6372),
 ('u', 6204),
 ('i', 5824),
 ('ur', 4956),
 ('at', 4873),
 ('threw', 3600),
 ('your', 3416),
 ('no', 3104),
 ('shit', 3101),
 ('hair', 3004),
 ('some', 2953),
 ('the', 2916),
 ('be', 2665),
 ('are', 2608),
 ('mah', 2600),
 ('was', 2577),
 ('then', 2572),
 ('would', 2534),
 ('if', 2532),
 ('say', 2519),
 ('head', 2501),
 ('girl', 2467),
 ('people', 2456),
 ('UR', 2453),
 ('stfu', 2451),
 ('make', 2443),
 ('good', 2435),
 ('saw', 2425),
 ('fat', 2424),
 ('mad', 2414),
 ('kno', 2413),
 ('sense', 2407),
 ('of;', 2402),
 ('named', 2401),
 ('corndog', 2400),
 ('CORNDOG', 2400),
 ('GOOR', 2400),
 ('twinkie', 2400),
 ('bitch', 1795),
 ('bich', 1456),
 ('her', 1364),
 ("didn't", 1224),
 ("did'nt", 1220),
 ('thru', 1200),
 ('off', 1047),
 ('fair', 961),
 ('phair', 960),
 ('phare', 960),
 ('fare', 960),
 ('faire', 960),
 ('ass', 853),
 ('lick', 807),
 ('lik', 804),
 ('lic', 803),
 ('on', 795),
 ('hoe', 771),
 ('so', 766),
 ('pussy', 727),
 ('love', 685),
 ('too', 638

In [45]:
# negative_counts.most_common()
print("Pos-to-neg ratio for 'the' = {}".format(pos_neg_ratios["the"]))
print("Pos-to-neg ratio for 'amazing' = {}".format(pos_neg_ratios["amazing"]))
print("Pos-to-neg ratio for 'shit' = {}".format(pos_neg_ratios["shit"]))

Pos-to-neg ratio for 'the' = 2.461319014418187
Pos-to-neg ratio for 'amazing' = 5.746203190540153
Pos-to-neg ratio for 'shit' = -2.5972980568759496


### What are the most common words for 'threats' and 'non-threats' 

In [46]:
pos_neg_ratios.most_common()

[('rock', 9.3969859002501916),
 ('badr', 8.7021778656296753),
 ('spend', 8.4314174143948328),
 ('Baby', 8.4213428657594029),
 ('glue', 8.4144957931778954),
 ('bed', 8.1028891346408685),
 ('Nicki', 8.0100275284817339),
 ('andr', 8.009363076630045),
 ('Young', 8.009363076630045),
 ('bakr', 8.009363076630045),
 ('rokr', 8.0090306850697299),
 ('love;', 7.640603826393634),
 ('money', 7.4073177104694174),
 ('gorgeous', 7.3895639536776354),
 ('walk', 7.3821243657375124),
 ('running', 7.3777589082278725),
 ('spoke', 7.3740018593501606),
 ('cold', 7.3607399030582776),
 ('hold', 7.3581937527330323),
 ('along', 7.3581937527330323),
 ('photo', 7.357556200910353),
 ('rest', 7.3562798765507482),
 ('room', 7.3485875309275928),
 ('race', 7.3466551631765391),
 ('lover', 7.3453648404168685),
 ('attractive', 7.3401868353201154),
 ('fly', 7.3401868353201154),
 ('sing', 7.333676395657684),
 ('lights', 7.3297496890415124),
 ('double', 7.329093736246592),
 ('Michael', 7.3251489579555749),
 ('annoys', 7.32514

In [47]:
list(reversed(pos_neg_ratios.most_common()))[0:30]

[('ol', -4.6051701859880918),
 ('matta', -4.6051701859880918),
 ('dawk', -4.6051701859880918),
 ('wiht', -4.6051701859880918),
 ('priscilla', -4.6051701859880918),
 ('prob', -4.6051701859880918),
 ('hare', -4.6051701859880918),
 ('twinkie', -4.6051701859880918),
 ('haire', -4.6051701859880918),
 ('GOOR', -4.6051701859880918),
 ('CORNDOG', -4.6051701859880918),
 ('corndog', -4.6051701859880918),
 ('hopp', -4.6051701859880918),
 ('alexis;', -4.6051701859880918),
 ('ghetto', -4.6051701859880918),
 ('threw', -4.2969425047274195),
 ('otha', -4.2388078038936694),
 ('stfu', -4.2067992413293345),
 ('bea', -4.1653125479201716),
 ('beah', -4.1653125479201716),
 ('yao', -4.1531850622450337),
 ('yow', -4.1531850622450337),
 ('yau', -4.1531850622450337),
 ('thru', -4.1459446655796457),
 ('mutha', -4.1047885213831838),
 ('kus', -3.9913597476646672),
 ('fade', -3.9753926193607354),
 ('UR', -3.9439031633388089),
 ('wdf', -3.9169857947702749),
 ('ah;', -3.9169857947702749)]

### Yeah... doesn't look like there is a pattern here... 
Maybe some more work on the pre-treatment is needed 

### Let's take a look 

In [48]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [49]:
hist, edges = np.histogram(list(map(lambda x:x[1],pos_neg_ratios.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Word Positive/Negative Affinity Distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [50]:
frequency_frequency = Counter()

for word, cnt in total_counts.most_common():
    frequency_frequency[cnt] += 1

In [51]:
hist, edges = np.histogram(list(map(lambda x:x[1],frequency_frequency.most_common())), density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="The frequency distribution of the words in our corpus")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [52]:
vocab = set(total_counts.keys())
vocab_size = len(vocab)
print(vocab_size)

19794


## Let's build a fully connected network to do this job

In [53]:
cyber_fc = fc_nn.CyberbullyingFullyConnectedNetwork(reviews=questions_answers, labels=labels,hidden_nodes=300,learning_rate=0.01)

In [54]:
cyber_fc.train(questions_answers, labels)

Progress:0.0% Speed(reviews/sec):0.0 #Correct:0 #Trained:1 Training Accuracy:0.0%
Progress:6.77% Speed(reviews/sec):5621. #Correct:2339 #Trained:2501 Training Accuracy:93.5%
Progress:13.5% Speed(reviews/sec):5715. #Correct:4784 #Trained:5001 Training Accuracy:95.6%
Progress:20.3% Speed(reviews/sec):5891. #Correct:7077 #Trained:7501 Training Accuracy:94.3%
Progress:27.1% Speed(reviews/sec):5981. #Correct:9484 #Trained:10001 Training Accuracy:94.8%
Progress:33.8% Speed(reviews/sec):5573. #Correct:11828 #Trained:12501 Training Accuracy:94.6%
Progress:40.6% Speed(reviews/sec):4853. #Correct:14262 #Trained:15001 Training Accuracy:95.0%
Progress:47.4% Speed(reviews/sec):3982. #Correct:16723 #Trained:17501 Training Accuracy:95.5%
Progress:54.2% Speed(reviews/sec):4169. #Correct:19028 #Trained:20001 Training Accuracy:95.1%
Progress:61.0% Speed(reviews/sec):4206. #Correct:21467 #Trained:22501 Training Accuracy:95.4%
Progress:67.7% Speed(reviews/sec):4219. #Correct:23901 #Trained:25001 Training 

In [55]:
len(cyber_fc.word2index)

15643

## P-CNN section 

In [56]:
import spacy # nl processing
nlp = spacy.load('en')

In [57]:
import util.Data as util_data
sents_encoder = util_data.reviews_labels_encoder(
    mw, 
    n_words_in_review = 10, 
    reviews = list(map(lambda x: util_data.review(x), questions_answers)),  
    labels = list(map(lambda x: util_data.label(x), labels)),
    spacy_nlp = nlp)

### Let's create network 

In [58]:
cnn_k = CyberbullyingDetectionnNN(features_in_words=300, words_in_review=10)
cnn_k.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 10, 300, 1)    0                                            
____________________________________________________________________________________________________
conv2D_1x1 (Conv2D)              (None, 10, 300, 100)  200                                          
____________________________________________________________________________________________________
conv2D_2x2 (Conv2D)              (None, 10, 300, 100)  500                                          
____________________________________________________________________________________________________
conv2D_3x3 (Conv2D)              (None, 10, 300, 100)  1000                                         
___________________________________________________________________________________________

### If you need sanity checking, run next cell

In [61]:
nn.sanity_check()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 10, 300, 1)    0                                            
____________________________________________________________________________________________________
conv2D_1x1 (Conv2D)              (None, 10, 300, 100)  200                                          
____________________________________________________________________________________________________
conv2D_2x2 (Conv2D)              (None, 10, 300, 100)  500                                          
____________________________________________________________________________________________________
conv2D_3x3 (Conv2D)              (None, 10, 300, 100)  1000                                         
___________________________________________________________________________________________

In [62]:
reviews_as_matrix = sents_encoder.reviews_as_matrix()

In [63]:
reviews_as_matrix.shape

(36885, 10, 300)

In [64]:
labels_as_matrix = sents_encoder.labels_as_matrix()

In [65]:
labels_as_matrix.shape

(36885, 2)

### Split the data
TODO: the datasets are very imbalanced - so we must do some over(/under) sampling. Maybe https://github.com/scikit-learn-contrib/imbalanced-learn ?  

In [71]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(reviews_as_matrix, labels_as_matrix, test_size=0.33, random_state=42)

In [66]:
cnn_k.fit(x_train=reviews_as_matrix.reshape(reviews_as_matrix.shape + (1,)), y_train=labels_as_matrix, batch_size=16, epochs=5)

Epoch 1/5
 1264/36885 [>.............................] - ETA: 1284s - loss: 0.0099 - acc: 0.9953

KeyboardInterrupt: 